In [16]:
import sys
import os

sys.path.append(os.path.abspath(".."))

from final_code.pubmed_esearch import get_pmcids
from final_code.pubmed_esearch import create_df_pmcids
import pandas as pd
import requests
import time
import xml.etree.ElementTree as ET

In [24]:
search_terms = ["Avian influenza outbreak", "EHEC outbreak"]

df = create_df_pmcids(search_terms=search_terms, no_of_results=50)
df["PMCID"] = "PMC" + df["PMCID"].astype(str)
df.head()

,PMCID,search_term
0,PMC11927218,Avian influenza outbreak
1,PMC11927074,Avian influenza outbreak
2,PMC11926602,Avian influenza outbreak
3,PMC11922848,Avian influenza outbreak
4,PMC11922521,Avian influenza outbreak


In [ ]:
def check_if_pmcid_is_available(pmcid):
    search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    params = {
        "db": "pmc", 
        "id": "open access[filter]+" + pmcid, 
        "retmode": "xml"
    }
    response = requests.get(search_url, params=params)
    time.sleep(0.5)
    if response.status_code == 200:
        root = ET.fromstring(response.content)

        for reply in root.findall('.//Reply'):
            if 'error' in reply.attrib and "The following PMCID is not available" in reply.attrib['error']:
               return False 

        return True
    else:
        return response.status_code

# u = check_if_pmcid_is_available("PMC11900536")
# print(u)
   

In [26]:
df["has_result"] = df["PMCID"].apply(check_if_pmcid_is_available)

df

,PMCID,search_term,has_result
0,PMC11927218,Avian influenza outbreak,True
1,PMC11927074,Avian influenza outbreak,False
2,PMC11926602,Avian influenza outbreak,False
3,PMC11922848,Avian influenza outbreak,True
4,PMC11922521,Avian influenza outbreak,True
...,...,...,...
95,PMC11758462,EHEC outbreak,True
96,PMC11746998,EHEC outbreak,True
97,PMC11749759,EHEC outbreak,True
98,PMC11719993,EHEC outbreak,True


In [27]:
df.to_csv("check_for_results.csv")

In [31]:
def get_call_link(pmcid):
    search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    params = {
        "db": "pmc", 
        "id": "open access[filter]+" + pmcid, 
        "retmode": "xml"
    }
    response = requests.get(search_url, params=params)
    time.sleep(0.5)

    return response.url

u = get_call_link("PMC11719993")
print(u)
   

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=open+access%5Bfilter%5D%2BPMC11719993&retmode=xml
